In [1]:
from objects import ingestable, transformable, spark_session
import os

Importing objects modules
Importing utils modules


In [2]:
source_name = "bank"
source_format = "csv"

In [3]:
bank_source = ingestable.ingestable(source_name, os.path.join(os.getcwd(), "..", "data", "*"), source_format)

In [4]:
print(bank_source)

ingestable(bank,C:\Users\juanl\OneDrive\Documentos\2-Workspaces\ETL\src\..\data\*,csv)


In [5]:
spark = spark_session.SingletonSparkSession().get_spark()

In [6]:
spark

In [7]:
bank_source.ingest(spark)

Processed rows: 12000


In [8]:
landing_df = spark.read.parquet(bank_source.get_landing_directory())
landing_df.show()  # Muestra algunas filas de la tabla
print("Ingested rows " + str(landing_df.count()))
landing_df.printSchema()  # Muestra el esquema de la tabla

+--------------+----------------+-------+----------------+-------------+--------------+------------+-------------------+--------+--------------------+--------------------+
|transaction_id|transaction_date| amount|transaction_type|merchant_name|      category|account_type|           location|currency|         description|     processing_time|
+--------------+----------------+-------+----------------+-------------+--------------+------------+-------------------+--------+--------------------+--------------------+
|             1|       8/23/2022|4792.59|         deposit|     Gigaclub|     groceries|    checking|           Lincheng|     CNY|venenatis lacinia...|2025-02-27 22:32:...|
|             2|        8/6/2022|9866.72|         deposit|      Jaxbean|transportation|    checking|       Wierzchucino|     PLN|varius ut blandit...|2025-02-27 22:32:...|
|             3|       8/21/2022|7312.82|      withdrawal|      Jaxspan| entertainment|     savings|              Sondo|     IDR|luctus tinc

In [9]:
bronze_transform = transformable.transformable(source_name, landing_df, "bronze")
bronze_transform.run()

In [10]:
bronze_df = spark.read.parquet(bronze_transform.get_target_directory())
bronze_df.show()  # Muestra algunas filas de la tabla
print("Ingested rows " + str(bronze_df.count()))
bronze_df.printSchema()  # Muestra el esquema de la tabla

+--------------+----------------+-------+----------------+-------------+--------------+------------+-------------------+--------+--------------------+--------------------+
|transaction_id|transaction_date| amount|transaction_type|merchant_name|      category|account_type|           location|currency|         description|     processing_time|
+--------------+----------------+-------+----------------+-------------+--------------+------------+-------------------+--------+--------------------+--------------------+
|             1|       8/23/2022|4792.59|         deposit|     Gigaclub|     groceries|    checking|           Lincheng|     CNY|venenatis lacinia...|2025-02-27 22:32:...|
|             2|        8/6/2022|9866.72|         deposit|      Jaxbean|transportation|    checking|       Wierzchucino|     PLN|varius ut blandit...|2025-02-27 22:32:...|
|             3|       8/21/2022|7312.82|      withdrawal|      Jaxspan| entertainment|     savings|              Sondo|     IDR|luctus tinc

In [11]:
silver_transform = transformable.transformable(source_name, bronze_df, "silver")
silver_transform.run()

In [12]:
silver_df = spark.read.parquet(silver_transform.get_target_directory())
silver_df.show()  # Muestra algunas filas de la tabla
print("Ingested rows " + str(silver_df.count()))
silver_df.printSchema()  # Muestra el esquema de la tabla

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/juanl/OneDrive/Documentos/2-Workspaces/ETL/silver/bank.

In [ ]:
bank_source.clear_checkpoint()